In [1]:
import pandas as pd
import numpy as np
import json
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pickle
from tqdm import tqdm 
from ast import literal_eval
import cobra
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
from functools import partial 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MaxNLocator, FormatStrFormatter

import gurobipy
gurobipy.setParam('OutputFlag', 0)
import sys
# sys.path.append('../Script/')
from common import *

# input and output

In [2]:
###input
# yeast870_path =  '../../Data/model/yeast-GEM.yml'
yeast8U_del_path = '../../Data/model/yeast8U_del_re.yml'
yeast870_path =  '../../Data/model/yeast-GEM.yml'
YMDB_total_smiles_path = '../../Data/database/ymdb/ymdb_final_result.xlsx'
ess_del_reaction_df_file = '../../Results/analysis/ess_del_reaction_df_re.csv'
biolog_del_reaction_df_file = '../../Results/analysis/Biolog_del_reaction_re.csv'

cut_off = 0.3
num = 5
num_process = 50
cut_off_path = f'../../Results/not_lipid/top{num}_{cut_off}_re/'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
# rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'
rxndb_total_info_to_model_del_reaction_path = '../../Results/not_lipid/top50_0.3_re/rxndb_total_info_to_model_del_reaction_top50_0.3.csv'

###output
rxndb_total_info_to_model_del2_reaction_path = '../../Results/not_lipid/top50_0.3_re/rxndb_total_info_to_model_del2_reaction_top50_0.3.csv'
metabolites_info_to_GEM_del_path = '../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_del_top50_0.3.csv'
yeast8U_del2_path = '../../Data/model/Yeast-MetaTwin.yml'

In [3]:
del_reaction_lst = []

ess_del_reaction_df = pd.read_csv(ess_del_reaction_df_file,index_col=None)
ess_del_reaction_df['reaction'] = ess_del_reaction_df['reaction'].apply(lambda x : literal_eval(x))
ess_del_reaction_df = ess_del_reaction_df[ess_del_reaction_df['num']<20]

for i in ess_del_reaction_df['reaction'].to_list():
    del_reaction_lst += i

print(len(del_reaction_lst))
del_reaction_lst = list(set(del_reaction_lst))
print(len(del_reaction_lst))

print('==============================')

biolog_del_reaction_df = pd.read_csv(biolog_del_reaction_df_file,index_col=None)
biolog_del_reaction_df['reaction'] = biolog_del_reaction_df['reaction'].apply(lambda x : literal_eval(x))
biolog_del_reaction_df = biolog_del_reaction_df[biolog_del_reaction_df['num']<20]

for i in biolog_del_reaction_df['reaction'].to_list():
    del_reaction_lst += i

print(len(del_reaction_lst))
del_reaction_lst = list(set(del_reaction_lst))
print(len(del_reaction_lst))

44
35
103
69


In [7]:
rxndb_total_info_to_model = pd.read_csv(rxndb_total_info_to_model_del_reaction_path)
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model = rxndb_total_info_to_model[~rxndb_total_info_to_model['NO'].isin(del_reaction_lst)]
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_del2_reaction_path, index=False)

(4801, 8)
(4732, 8)


# met_info add YMDBID and YMDBname

In [8]:
met_info = pd.read_csv(metabolites_info_to_GEM_path)
met_info = met_info[~met_info['ID'].str.contains('s_')]
YMDB_Data = pd.read_excel(YMDB_total_smiles_path)
YMDB_Data = YMDB_Data[YMDB_Data['in_model']==0]

def process_metinfoID(metinfoID, met_info, YMDB_Data):
    metinfoID_smiles = met_info[met_info['ID'] == metinfoID]['new_met_smiles'].to_list()[0]
    YMDB_ID = []
    for index, row in YMDB_Data.iterrows():
        if compare_smiles_inchikey(metinfoID_smiles, row['SMILES']):
            YMDB_ID.append(row['ID'])
    return metinfoID, YMDB_ID

metinfoID_2_YMDBID = {}
metinfoID_list = list(set(met_info['ID'].to_list()))
process_partial = partial(process_metinfoID, met_info=met_info, YMDB_Data=YMDB_Data)
with mp.Pool(60) as pool:
    results = list(tqdm(pool.imap_unordered(process_partial, metinfoID_list), total=len(metinfoID_list)))
for metinfoID, YMDB_ID in results:
    metinfoID_2_YMDBID[metinfoID] = YMDB_ID

100%|██████████| 418/418 [05:48<00:00,  1.20it/s]


# reset rxnID

In [9]:
rxndb_to_model = pd.read_csv(rxndb_total_info_to_model_del2_reaction_path)
rxndb_to_model['equation_dict'] = rxndb_to_model['equation_dict'].apply(lambda x:literal_eval(x))
rxndb_to_model = rxndb_to_model[~rxndb_to_model['equation'].str.startswith(' =>')]
rxndb_to_model = rxndb_to_model[~rxndb_to_model['equation'].str.endswith(' => ')]
rxndb_to_model = rxndb_to_model.drop_duplicates(['equation'], keep='first')
with open(yeast8_reaction_in_rxndb_json, 'r') as file:
    yeast8_reaction_in_rxndb_dict = json.load(file)
yeast8_reaction_in_rxndb = list(set([x for lst in yeast8_reaction_in_rxndb_dict.values() for x in lst]))
rxndb_to_model = rxndb_to_model[~rxndb_to_model['NO'].isin(yeast8_reaction_in_rxndb)]
rxndb_to_model = rxndb_to_model.reset_index(drop=True)
rxndb_to_model['NO'] = 'rxn' + (rxndb_to_model.index + 1).astype(str)
rxndb_to_model.to_csv(rxndb_total_info_to_model_del2_reaction_path,index=None)
rxndb_to_model.head()

,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxn1,1.14.13,MNXR149799,CCCCCCCCCCCC(=O)[O-].O=O>>CCCCCCCCCC(O)CC(=O)[...,CCCCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_1065': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1065 + s_1275 + s_0794 + s_1212 => sn_1...
1,rxn2,1.14.13,MNXR149799,CCCCCCCCCC(=O)[O-].O=O>>CCCCCCCC(O)CC(=O)[O-].O,CCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_0595': -1, 's_1275': -1, 's_0794': -1, 's_...",s_0595 + s_1275 + s_0794 + s_1212 => sn_1...
2,rxn3,1.14.13,MNXR149799,CCCCCCCC(=O)[O-].O=O>>CCCCCC(O)CC(=O)[O-].O,CCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]2O...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_1248': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1248 + s_1275 + s_0794 + s_1212 => sn_1...
3,rxn4,"4.3.1, 3.5.99",MNXR114398,CCC(=[NH2+])C(=O)[O-].O>>CCC(=O)C(=O)[O-].[NH4+],CCC(=[NH2+])C(=O)[O-].O>>CCC(=O)C(=O)[O-].[NH4+],(YGL196W) or (YER086W) or (YCL064C) or (YKL218...,"{'sn_307': -1, 's_0803': -1, 's_0178': 1, 's_0...",sn_307 + s_0803 => s_0178 + s_0419
4,rxn5,"4.3.1, 3.5.99",MNXR114398,O.[NH2+]=C(CC(=O)[O-])C(=O)[O-]>>O=C([O-])CC(=...,O.[NH2+]=C(CC(=O)[O-])C(=O)[O-]>>O=C([O-])CC(=...,(YGL196W) or (YER086W) or (YCL064C) or (YKL218...,"{'s_0803': -1, 'sn_321': -1, 's_1271': 1, 's_0...",s_0803 + sn_321 => s_1271 + s_0419


,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxn1,1.14.13,MNXR149799,CCCCCCCCCCCC(=O)[O-].O=O>>CCCCCCCCCC(O)CC(=O)[...,CCCCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_1065': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1065 + s_1275 + s_0794 + s_1212 => sn_1...
1,rxn2,1.14.13,MNXR149799,CCCCCCCCCC(=O)[O-].O=O>>CCCCCCCC(O)CC(=O)[O-].O,CCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_0595': -1, 's_1275': -1, 's_0794': -1, 's_...",s_0595 + s_1275 + s_0794 + s_1212 => sn_1...
2,rxn3,1.14.13,MNXR149799,CCCCCCCC(=O)[O-].O=O>>CCCCCC(O)CC(=O)[O-].O,CCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]2O...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_1248': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1248 + s_1275 + s_0794 + s_1212 => sn_1...
3,rxn4,"4.3.1, 3.5.99",MNXR114398,CCC(=[NH2+])C(=O)[O-].O>>CCC(=O)C(=O)[O-].[NH4+],CCC(=[NH2+])C(=O)[O-].O>>CCC(=O)C(=O)[O-].[NH4+],(YGL196W) or (YER086W) or (YCL064C) or (YKL218...,"{'sn_307': -1, 's_0803': -1, 's_0178': 1, 's_0...",sn_307 + s_0803 => s_0178 + s_0419
4,rxn5,"4.3.1, 3.5.99",MNXR114398,O.[NH2+]=C(CC(=O)[O-])C(=O)[O-]>>O=C([O-])CC(=...,O.[NH2+]=C(CC(=O)[O-])C(=O)[O-]>>O=C([O-])CC(=...,(YGL196W) or (YER086W) or (YCL064C) or (YKL218...,"{'s_0803': -1, 'sn_321': -1, 's_1271': 1, 's_0...",s_0803 + sn_321 => s_1271 + s_0419
...,...,...,...,...,...,...,...,...
3325,rxn3326,2.7.1,MNXR173125_reverse,O=CC(O)C(O)C(O)C(O)CO.O=P([O-])([O-])OCC1OC(OC...,O=CC(O)C(O)C(O)C(O)CO.O=P([O-])([O-])OCC1OC(OC...,(YDR051C) or (YFR053C) or (YOL136C) or (YGR205...,"{'sn_71': -1, 'sn_28': -1, 's_3998': 1, 's_056...",sn_71 + sn_28 => s_3998 + s_0568
3326,rxn3327,2.7.1,MNXR173125_reverse,O=C([O-])C(O)C(O)C(O)C(O)CO.O=P([O-])([O-])OCC...,O=C([O-])C(O)C(O)C(O)C(O)CO.O=P([O-])([O-])OCC...,(YDR051C) or (YFR053C) or (YOL136C) or (YGR205...,"{'s_3875': -1, 'sn_28': -1, 's_3998': 1, 's_03...",s_3875 + sn_28 => s_3998 + s_0340
3327,rxn3328,2.7.1,MNXR173125_reverse,O=C([O-])C(=O)C(O)C(O)C(O)CO.O=P([O-])([O-])OC...,O=C([O-])C(=O)C(O)C(O)C(O)CO.O=P([O-])([O-])OC...,(YDR051C) or (YFR053C) or (YOL136C) or (YGR205...,"{'sn_245': -1, 'sn_28': -1, 's_3998': 1, 'sn_9...",sn_245 + sn_28 => s_3998 + sn_95
3328,rxn3329,2.7.1,MNXR173125_reverse,O=C([O-])C(O)CO.O=P([O-])([O-])OCC1OC(OC2(CO)O...,O=C([O-])C(O)CO.O=P([O-])([O-])OCC1OC(OC2(CO)O...,(YDR051C) or (YFR053C) or (YOL136C) or (YGR205...,"{'s_3958': -1, 'sn_28': -1, 's_3998': 1, 's_02...",s_3958 + sn_28 => s_3998 + s_0260


In [7]:
get_yeast8U_step4(metabolites_info_to_GEM_path,rxndb_total_info_to_model_del2_reaction_path,yeast870_path,yeast8U_del2_path,metinfoID_2_YMDBID)

(4151, 4)
(3330, 8)


100%|██████████| 3330/3330 [00:13<00:00, 247.24it/s]


In [8]:
yeast8 = cobra.io.load_yaml_model(yeast870_path)
yeast8.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.5858,0,0.00%
s_0565,r_1714,1,6,100.00%
s_0796,r_1832,0.04144,0,0.00%
s_0925,r_1861,2.491E-06,0,0.00%
s_1277,r_1992,2.332,0,0.00%
s_1324,r_2005,0.5331,0,0.00%
s_1374,r_2020,0.0002974,0,0.00%
s_1438,r_2049,0.0003253,0,0.00%
s_1468,r_2060,0.007047,0,0.00%
s_4200,r_4593,0.0001057,0,0.00%


In [9]:
yeast8U_mod = cobra.io.load_yaml_model(yeast8U_del2_path)

In [10]:
yeast8U_mod.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.6402,0,0.00%
s_0565,r_1714,1,6,100.00%
s_0796,r_1832,0.04529,0,0.00%
s_0925,r_1861,0.005913,0,0.00%
s_1277,r_1992,1.997,0,0.00%
s_1324,r_2005,0.04482,0,0.00%
s_1374,r_2020,0.000325,0,0.00%
s_1438,r_2049,0.0003555,0,0.00%
s_1468,r_2060,0.001791,0,0.00%
s_4200,r_4593,0.0001155,0,0.00%


In [11]:
rxndb_to_model = pd.read_csv(rxndb_total_info_to_model_del2_reaction_path)
metabolites_info_to_GEM = pd.read_csv(metabolites_info_to_GEM_path)
met_id_lst = []
rxndb_to_model['equation_dict'] = rxndb_to_model['equation_dict'].apply(lambda x:literal_eval(x))
for i in range(len(rxndb_to_model)):
    met_id_lst += list(rxndb_to_model['equation_dict'].iloc[i].keys())

met_id_lst = list(set(met_id_lst))
print(metabolites_info_to_GEM.shape)
metabolites_info_to_GEM_del = metabolites_info_to_GEM[metabolites_info_to_GEM['ID'].isin(met_id_lst)]
print(metabolites_info_to_GEM_del.shape)
metabolites_info_to_GEM_del.to_csv(metabolites_info_to_GEM_del_path, index=False)

(4151, 4)
(3465, 4)
